# Elkin Cylinder 3D Visualization Notebook

This notebook provides an **interactive 3D visualization** of Elkin's cylinder construction for 3-AP-free sets (Elkin, 2011).

It includes full enumeration for small N, GPU-friendly sampling for huge sets, and customizable color palettes for cylinder layers.

## Features
- Interactive 3D plotting using Plotly
- Hover info for each point (coordinates, layer)
- Color-coded cylinder layers with selectable palettes
- Automatic rotation (optional) for better visualization
- Sliders for `N`, `d1`, `d2`, and `Sample Ratio`
- Exportable to standalone HTML
- Full enumeration for small sets, sampling for large sets


In [ ]:
# ---------------------------
# REQUIREMENTS
# ---------------------------
!
pip install plotly numpy ipywidgets math json

In [ ]:
# ---------------------------
# IMPORTS
# ---------------------------
import math
import random
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
from typing import List, Tuple


In [ ]:
# ---------------------------
# ELKIN CYLINDER CONSTRUCTION
# ---------------------------
def optimize_dimensions(N: int) -> Tuple[int,int,int]:
    logN = math.log(N) if N > 1 else 1
    loglogN = math.log(logN) if logN > 1 else 1
    d_total = max(int(math.floor(math.sqrt(logN / math.log(2)))),3)
    d2 = max(1, min(d_total-1, int(math.ceil(loglogN**0.25))))
    d1 = d_total - d2
    return d_total,d1,d2

def generate_elkin_cylinder(N: int, d1:int, d2:int, seed:int=42, sample_ratio:float=1.0) -> Tuple[np.ndarray,np.ndarray]:
    random.seed(seed)
    d_total = d1 + d2
    M = max(3,int(math.ceil(N**(1/d_total))))
    mid = M // 2

    # Sphere part in d1 dimensions
    max_radius = d1*(mid-1)**2
    squares = [(i-mid)**2 for i in range(M)]
    dp = [0]*(max_radius+1)
    dp[0]=1
    for _ in range(d1):
        new_dp = [0]*(max_radius+1)
        for r in range(max_radius+1):
            if dp[r]==0: continue
            for sq in squares:
                if r+sq <= max_radius:
                    new_dp[r+sq]+=dp[r]
        dp=new_dp
    R2 = max(range(max_radius+1), key=lambda r: dp[r])

    # Generate points
    points=[]
    def backtrack_sphere(dim, current_sum, current_vector):
        if dim==d1:
            if current_sum==R2:
                extend_cylinder(current_vector)
            return
        remaining = d1 - dim
        max_sq=(mid-1)**2
        if current_sum > R2 or current_sum + remaining*max_sq < R2:
            return
        for x in range(M):
            y = x - mid
            y2 = y*y
            backtrack_sphere(dim+1, current_sum+y2, current_vector+[(x, M**(d_total-dim-1))])

    def extend_cylinder(sphere_vectors):
        base_num = sum(digit*place for digit,place in sphere_vectors)
        free_positions = [M**(d_total-(d1+i)-1) for i in range(d2)]
        total_free = M**d2
        for free_index in range(total_free):
            num = base_num
            temp = free_index
            for place in free_positions:
                digit=temp%M
                num+=digit*place
                temp//=M
            if num<N: points.append(num)

    backtrack_sphere(0,0,[])

    points_arr = np.array(points)
    if sample_ratio<1.0:
        sample_size = max(1,int(len(points_arr)*sample_ratio))
        points_arr = np.random.choice(points_arr,sample_size,replace=False)

    # Compute coordinates for 3D visualization
    coords = np.zeros((len(points_arr),3))
    for i, val in enumerate(points_arr):
        coords[i,0] = val % M
        coords[i,1] = (val//M) % M
        coords[i,2] = (val//(M*M)) % M

    return points_arr, coords

In [ ]:
# ---------------------------
# 3D PLOT FUNCTION
# ---------------------------
def plot_elkin_3d(coords: np.ndarray, color_palette:str='Viridis', auto_rotate:bool=False):
    fig = go.Figure(data=[
        go.Scatter3d(
            x=coords[:,0],
            y=coords[:,1],
            z=coords[:,2],
            mode='markers',
            marker=dict(
                size=3,
                color=coords[:,2],
                colorscale=color_palette,
                opacity=0.8,
                colorbar=dict(title='Layer (Z)')
            ),
            text=[f'X={x}, Y={y}, Z={z}' for x,y,z in coords],
            hoverinfo='text'
        )
    ])
    fig.update_layout(scene=dict(aspectmode='cube'))
    if auto_rotate:
        fig.update_layout(updatemenus=[dict(type='buttons',buttons=[dict(label='Play',method='animate',args=[None])])])
    fig.show()

In [ ]:
# ---------------------------
# INTERACTIVE WIDGETS
# ---------------------------
def interactive_elkin():
    def plot(N=1000,d1=3,d2=1,sample_ratio=1.0,color_palette='Viridis',auto_rotate=False):
        _, coords = generate_elkin_cylinder(N,d1,d2,sample_ratio=sample_ratio)
        plot_elkin_3d(coords,color_palette=color_palette,auto_rotate=auto_rotate)
    
    interact(plot,
             N=IntSlider(min=10,max=10000,step=10,value=1000,description='N'),
             d1=IntSlider(min=1,max=10,step=1,value=3,description='d1'),
             d2=IntSlider(min=0,max=5,step=1,value=1,description='d2'),
             sample_ratio=FloatSlider(min=0.01,max=1.0,step=0.01,value=1.0,description='Sample Ratio'),
             color_palette=Dropdown(options=['Viridis','Plasma','Turbo','Cividis'],value='Viridis',description='Palette'),
             auto_rotate=Dropdown(options=[False,True],value=False,description='Auto Rotate')
            )

interactive_elkin()

## Usage Notes
- Open this notebook in **Jupyter Notebook** or **JupyterLab**.
- Run all cells in order.
- Use the sliders to adjust:
  - **N**: total number of points
  - **d1**: constrained (sphere) dimensions
  - **d2**: free (cylinder) dimensions
  - **Sample Ratio**: fraction of points to display (GPU-friendly sampling for large N)
  - **Color Palette**: color scheme for layers
  - **Auto Rotate**: optional automatic rotation
- For small N, full enumeration is used.
- For huge N (>~100,000 points), sampling is used to keep browser responsive.
- To export to HTML, use File → Download as → HTML (.html)
- You can still analyze the full dataset in memory even if sampling is active for display.